In [1]:
import itertools
from collections import defaultdict
import os
import pickle
import multiprocessing
import pdb

import seaborn as sns
sns.set()
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import scipy
from scipy import stats
from statsmodels.stats.power import TTestIndPower
from tqdm import tqdm

from utils import filter_pairs, test_all

In [2]:
lp_source = 'X'
lp_target = 'X'
lengths = [600, 1200, 1800, 2400, 3000]

In [3]:
pairs = filter_pairs(lp_source, lp_target, size=1000)
len(pairs)

Annotated data loaded


1000

In [4]:
# pocock corrections
correction = 'pocock'
p = 0.05
pocock = [None, p, 0.0294, 0.0221, 0.0182, 0.0158]
bonferroni = [None, p/1, p/2, p/3, p/4, p/5]

In [5]:
def interim_run(x, steps=3, correction='pocock'):
    sys1, sys2, length, pvalue = x
    assert(pvalue == 0.05)
    
    if correction == 'pocock':
        p_0 = pocock[steps]
    elif correction == 'bonferroni':
        p_0 = bonferroni[steps]
    
    succeeded = 0

    # shuffle
    sys1_shuf = sys1.sample(length, replace=True)
    sys2_shuf = sys2.sample(length, replace=True)

    step_size = int(length/steps)
    for step in range(1, steps+1):
        size = step * step_size
        baseline_sys = sys1_shuf.head(size)
        other_sys = sys2_shuf.head(size)

        ind, pvalue = stats.mannwhitneyu(baseline_sys, other_sys)

        if pvalue < p_0:
            succeeded = 1
            break
            
    return succeeded, size

In [ ]:
for length in [600, 1200, 1800, 2400, 3000]:
    results = test_all(pairs, interim_run, length, 0.05)
    np.save(open('interim_%s_%s-%s_%d.npz' % (correction, lp_source, lp_target, length), 'wb'), results)

 11%|█         | 108/1000 [01:09<10:38,  1.40it/s]